### Evaluating Recommendation models on hour clustered data

In [2]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_836eb90819e64bfab21841eef5b3bf3c(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_6429e107b4727d3f5906de019f7fad88ada04fb8','domain': {'id': 'b5bbfbbc2d764a9096fd4aeb7d0d1477'},
            'password': 'o{a{0]Dnx^MJ-dc1'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)



In [16]:
normratings= pd.read_csv(get_object_storage_file_with_credentials_836eb90819e64bfab21841eef5b3bf3c('insta', 'hourcluster1.csv'))
normratings.head()


,Unnamed: 0,user_id,product_id,rating
0,0,2,32792,5.000000
1,1,2,47209,5.000000
2,2,2,1559,3.333333
3,3,2,7781,3.333333
4,4,2,16589,3.333333


In [3]:
!pip install --user --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.1/goel.t@husky.neu.edu/E914-C086-1338-7082-B278-E8A7-1C13-1FDE/GraphLab-Create-License.tar.gz
    

Requirement already up-to-date: graphlab-create>=2.1 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from GraphLab-Create-License==2.1)
Requirement already up-to-date: psclient in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from graphlab-create>=2.1->GraphLab-Create-License==2.1)
Requirement already up-to-date: boto==2.33.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from graphlab-create>=2.1->GraphLab-Create-License==2.1)
Requirement already up-to-date: decorator==4.0.9 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from graphlab-create>=2.1->GraphLab-Create-License==2.1)
Requirement already up-to-date: tornado==4.3 in /usr/local/src/bluemix_jupyter_bundle.v73/notebook/lib/python2.7/site-packages (from g

### Running Recommendations by clustering on basis of hour the products were purchase

In [17]:
import numpy as np
import graphlab

msk = np.random.rand(len(normratings)) < 0.8
train = normratings[msk]
test = normratings[~msk]
train_data = graphlab.SFrame(train)
test_data=graphlab.SFrame(test)

#### Creating item similarity model, popularity model and latent model on the clustered data for comparing and selecting best out of them

In [18]:
item_sim_model = graphlab.item_similarity_recommender.create(train_data, user_id='user_id', item_id='product_id', target='rating', similarity_type='pearson')
popularity_model = graphlab.popularity_recommender.create(train_data, user_id='user_id', item_id='product_id', target='rating')
latent_model=graphlab.factorization_recommender.create(train_data, user_id='user_id', item_id='product_id', target='rating')
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model,latent_model])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model,latent_model])

Recsys training: model = item_similarity

Warning: Ignoring columns Unnamed: 0;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1766505 observations with 111712 users and 40992 items.

...

recommendations finished on 93000/96226 queries. users per second: 659.629

recommendations finished on 94000/96226 queries. users per second: 662.931

recommendations finished on 95000/96226 queries. users per second: 666.463

recommendations finished on 96000/96226 queries. users per second: 668.865


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.0108494585663  | 0.00243820258362 |
|   2    | 0.00785650447904 | 0.00360404695889 |
|   3    | 0.00654362299864 | 0.00459962148588 |
|   4    | 0.00570531872883 | 0.00533079189612 |
|   5    | 0.00517947332322 | 0.00607772321917 |
|   6    | 0.00475616430764 | 0.00670381618461 |
|   7    | 0.0044404393229  | 0.00731151345478 |
|   8    | 0.00416857190364 | 0.00786999940604 |
|   9    | 0.00398252262613 | 0.00849827116364 |
|   10   | 0.00382121256209 | 0.00912320325612 |
+--------+------------------+------------------+
[10 rows x 3 columns]

Model compare metric: precision_recall
Canvas is accessible via web browser at the URL: http://localhost:42953/index.html
Opening Canvas in default web browser.


###  As we can see that latent model which uses unsupervised learning to shrink the users traits into latent features gives us the best result

###  Below is the code that is happening in the library
  product_crosstab=pd.pivot_table(data=normratings,values='rating',index='user_id',columns='product_id',fill_value=0)

  X=product_crosstab.values.T
  
  import sklearn
  
  from sklearn.decomposition import TruncatedSVD

  SVD=TruncatedSVD(n_components=20,random_state=17)

  resultant_matrix=SVD.fit_transform(X)  
  
  resultant_matrix.shape
  
  import numpy as np
  
  corr_mat = np.corrcoef(resultant_matrix)
   
  product_ids=product_crosstab.columns
  
  productlist=list(product_ids)
  
  productindexlist=[]
  
 for i in productlist:
 
    productindexlist.append(productlist.index(i))
    
  allrecommendations=[]    
  
  for j in productindexlist:
  
    corr_j=corr_mat[j]
 
    corrlist=list(product_ids[(corr_j<1.0)&(corr_j>0.98)])
    
    corrlist.append(j)
  
    allrecommendations.append(corrlist)
  
  print(allrecommendations)

### We use the Graphlab because it was able to scale since it was able to utilize the ibm dsx spark clusters